In [40]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import os

## Counties

In [31]:
# Load fips codes from census
state_fips = (
    pd.read_csv(
        "data/state_fips.txt", 
        delimiter="|",
        dtype="str",
    )
    .rename(
        columns = {
            "STATE": "STATEFP"
        }
    )
)

state_fips.head(1)

,STATEFP,STUSAB,STATE_NAME,STATENS
0,01,AL,Alabama,01779775


In [32]:
# load county geojson
counties = gpd.read_file("data/counties-geojson.json").drop(
    columns = ["AFFGEOID", "GEOID", "LSAD", "ALAND", "AWATER"]
)

# find center of plot and flip y axis

b = counties.total_bounds

center = (
    (b[0] + b[2]/2), 
    (b[1] + b[3]/2)
)

# Flip y-axis of geometry column
counties['geometry'] = counties.scale(
     yfact = -1,
     origin = center
)

In [33]:
counties.head()

,id,STATEFP,COUNTYFP,COUNTYNS,NAME,geometry
0,04015,04,015,00025445,Mohave,"POLYGON ((172.6458858892796 310.210411054158, ..."
1,22105,22,105,00559500,Tangipahoa,"POLYGON ((594.0189129581094 151.4400278402344,..."
2,16063,16,063,00395624,Lincoln,"POLYGON ((191.0281654950103 452.5163668978029,..."
3,27119,27,119,00659505,Polk,"POLYGON ((473.1396811625132 534.2774157024993,..."
4,38017,38,017,01034226,Cass,"POLYGON ((464.3433755327998 513.7100319907453,..."


In [34]:
# make a destination folder
try:
    os.mkdir("output/counties")
except FileExistsError:
    pass

In [37]:
# merge geodataframe with fips file and write out 
(
    counties.merge(
        state_fips,
        how = "left",
        on = "STATEFP"
    )
    .rename(
        columns = {
            "STATEFP": "state_fips",
            "COUNTYFP": "county_fips",
            "COUNTYNS": "county_gnis",
            "NAME": "county_name",
            "STUSAB": "state_abv",
            "STATE_NAME": "state_name",
            "STATENS": "state_gnis"
        }
     )
    .to_file(
        "output/counties/counties.shp"
    )
)